In [1]:
import requests
import pandas as pd
import numpy as np
import mysql.connector

In [2]:
from geopy.geocoders import Nominatim

# Evaluación II
1. Utilizando la API extraed toda la información que podáis de ella. La url para hacer las llamadas es:
`API_URL = "http://universities.hipolabs.com/search?country=NOMBREPAIS"`


In [3]:
def llamar_API(lista_paises):
    
    """
    Acepta una lista de países y devuelve un dataframe de los datos de los países indicados
    obtenidos de la API de hipolabs sobre universidades.
    
    Parametros:
    self.lista_paises: una lista de paises
    
    Returns:
    un dataframe con los datos unidos de todos los paises indicados
    """
    
    dataframe = pd.DataFrame()
    
    for p in lista_paises:
        url = f"http://universities.hipolabs.com/search?country={p}"
        response = requests.get(url=url)
        status = response.status_code
        razon = response.reason
        
        if status == 200:
            print(f'La peticion al API para {p} se ha realizado con éxito.')
        else: 
            print(f'Respuesta {status}: {razon}')
        
        df_pais = pd.json_normalize(response.json())
    
        dataframe = pd.concat([dataframe, df_pais], axis=0)
            
    print(f'Sus datos ya están listos en un dataframe. Aquí tiene los primeros 3 filas:')
    
    display(dataframe.head(3))
    
    return dataframe

In [4]:
df = llamar_API(['Argentina', 'Canada', 'United States'])

La peticion al API para Argentina se ha realizado con éxito.
La peticion al API para Canada se ha realizado con éxito.
La peticion al API para United States se ha realizado con éxito.
Sus datos ya están listos en un dataframe. Aquí tiene los primeros 3 filas:


,state-province,domains,name,country,web_pages,alpha_two_code
0,Buenos Aires,[atlantida.edu.ar],Universidad Atlantida Argentina,Argentina,[http://www.atlantida.edu.ar/],AR
1,Buenos Aires,[austral.edu.ar],Universidad Austral Buenos Aires,Argentina,[http://www.austral.edu.ar/],AR
2,Ciudad Autónoma de Buenos Aires,[caece.edu.ar],"Universidad CAECE, Buenos Aires",Argentina,[http://www.caece.edu.ar/],AR


In [5]:
df['country'].value_counts()

United States    2281
Canada            154
Argentina          87
Name: country, dtype: int64

2. Una vez tengáis todos los datos de la API, deberéis realizar una serie de procesos de limpieza, estos incluyen:
- Cambiad los nombres de las columnas para homogeneizarlas, tenemos columnas que tienen - y otras _. Unifícalo para que todo vaya con _.
- La columna de domains nos da una información similar a la de web_pages. Eliminad la columna domains.

In [6]:

def limpieza_columnas(dataframe):
    
    """
    Acepta un dataframe y realiza dos tareas de limpieza sobre ello:
    - reemplaza los guiones por guiones bajos en los nombres de las columnas
    - elimina la columna de domains
    
    Parametros:
    - un dataframe de datos extraidos del API de universidades de hipolabs        
    
    Returns:
    - none
    """
    
    diccionario = {col : col.replace('-', '_') for col in dataframe.columns}

    dataframe.rename(columns=diccionario, inplace=True)
    
    dataframe.drop(columns='domains', axis=1, inplace=True)
    
    print("Su dataframe se ha limpiado. Aquí tiene la primera fila:")
    display(dataframe.head(1))

In [7]:
# llamo a la funcion
limpieza_columnas(df)

Su dataframe se ha limpiado. Aquí tiene la primera fila:


,state_province,name,country,web_pages,alpha_two_code
0,Buenos Aires,Universidad Atlantida Argentina,Argentina,[http://www.atlantida.edu.ar/],AR


3. Si exploramos la columna de web_pages, nos daremos cuenta que hay universidades, como por ejemplo la Universidad de "Cégep de Saint-Jérôme" de Canadá que en su columna de web_pages tiene más de un valor dentro de la lista. Esto es poco práctico y puede llegar a no tener sentido. el objetivo de este ejericio es que usando el método explode de pandas separéis cada elemento de la lista en una fila nueva.

In [8]:
df.shape

(2522, 5)

In [9]:
def explode_columna(dataframe, columna):
    '''
    Acepta un dataframe y columna y separa los valores de la columna que estan en listas, creando una fila por elemento
    
    Parametros:
    - un dataframe
    - string: nombre de una columna del dataframe
    
    Returns:
    - un dataframe
    '''
    return dataframe.explode(columna)
    

In [10]:
df = explode_columna(df, 'web_pages')

In [11]:
df.shape

(2535, 5)

4. Una vez hayáis realizado el explode, chequead si tenéis duplicados basándonos unicamente en el nombre de la universidad, en caso de que si, eliminandlos.

In [12]:
# chequeo si hay duplicados de la columna de los nombres de las universidades
df['name'].duplicated().sum()

28

In [13]:
def eliminar_dup(dataframe, columna):
    
    '''
    Acepta un dataframe y una columna y elimina las filas con valores duplicados de la columna indicada
    
    Parametros:
    - dataframe
    - string: nombre de una columna del dataframe
    '''
    
    num_dup = dataframe[columna].duplicated().sum()
    dataframe.drop_duplicates(subset=columna, inplace=True)
    
    print(f'Se han eliminado {num_dup} filas del dataframe.')

In [14]:
eliminar_dup(df, 'name')

# compruebo que el numero de filas ha cambiado
df.shape

Se han eliminado 28 filas del dataframe.


(2507, 5)

5. Si exploramos la columna de state_province veremos que hay universidades cuyo valor para esta columna es None. Cread una función para reemplazar los None por nulos de numpy.

In [15]:
df['state_province'].unique()

array(['Buenos Aires', 'Ciudad Autónoma de Buenos Aires', 'Entre Ríos',
       'Salta', 'Córdoba', 'Mendoza', 'Santa Fé', None,
       'Santiago Del Estero', 'Misiones', 'Catamarca', 'Formosa', 'Jujuy',
       'La Rioja', 'La Pampa', 'San Juan', 'San Luis', 'Tucumán',
       'Quebec', 'Ontario', 'Nova Scotia', 'British Columbia', 'Alberta',
       'Manitoba', 'New Brunswick', 'Saskatchewan',
       'Newfoundland and Labrador', 'Prince Edward Island', 'Yukon',
       'Pennsylvania', 'NV', 'Iowa', 'VA', 'TX', 'Colorado', 'IN', 'CA',
       'South Carolina', 'Washington', 'NY', 'Texas', 'ND', 'MI', 'Ohio',
       'Florida', 'California', 'North Carolina', 'Michigan', 'GA',
       'New York, NY'], dtype=object)

In [16]:
def none_nan(dataframe, columna):
    
    '''
    Acepta un dataframe y una columna y reemplaza cualquier valor de None por np.nan en 
    la columna indicada.
    
    Parametros:
    - dataframe
    - string: nombre de una columna del dataframe
    '''    
    
    num_none = dataframe[columna].isna().sum()
    
    dataframe[columna].fillna(value=np.nan, inplace=True)
    
    print(f'Se han reemplazado {num_none} valores None por np.nan en la columna {columna}.')

In [17]:
none_nan(df, 'state_province')

Se han reemplazado 2234 valores None por np.nan en la columna state_province.


In [18]:
# compruebo que ya no sale None en los valores unicos de la columna

df['state_province'].unique()

array(['Buenos Aires', 'Ciudad Autónoma de Buenos Aires', 'Entre Ríos',
       'Salta', 'Córdoba', 'Mendoza', 'Santa Fé', nan,
       'Santiago Del Estero', 'Misiones', 'Catamarca', 'Formosa', 'Jujuy',
       'La Rioja', 'La Pampa', 'San Juan', 'San Luis', 'Tucumán',
       'Quebec', 'Ontario', 'Nova Scotia', 'British Columbia', 'Alberta',
       'Manitoba', 'New Brunswick', 'Saskatchewan',
       'Newfoundland and Labrador', 'Prince Edward Island', 'Yukon',
       'Pennsylvania', 'NV', 'Iowa', 'VA', 'TX', 'Colorado', 'IN', 'CA',
       'South Carolina', 'Washington', 'NY', 'Texas', 'ND', 'MI', 'Ohio',
       'Florida', 'California', 'North Carolina', 'Michigan', 'GA',
       'New York, NY'], dtype=object)

6. Después del último cambio, os habréis dado cuenta que tenemos muchos valores nulos dentro de la columna de state_province, por lo que nuestro jefe nos pide que reemplacemos esos nulos por "Unknow". No nos piden ningún método especifico, asi que podremos usar el método que queramos.

In [19]:
def reemplazar_nulos(dataframe, columna, valor_nuevo):
    '''
    Acepta un dataframe y una columna y reemplaza cualquier valor nulo por el valor nuevo indicado en 
    la columna indicada.
    
    Parametros:
    - un dataframe
    - string: nombre de una columna del dataframe
    - string: valor nuevo que queremos en vez de nulos
    
    Returns:
    - none
    '''    
    num_none = dataframe[columna].isnull().sum()
    
    dataframe[columna].fillna(value=valor_nuevo, inplace=True)
    
    print(f'Se han reemplazado {num_none} valores nulos en la columna {columna} por {valor_nuevo}.')

In [20]:
reemplazar_nulos(df, 'state_province', 'Unknown')

df['state_province'].value_counts().head(5)

Se han reemplazado 2234 valores nulos en la columna state_province por Unknown.


Unknown             2234
Ontario               46
British Columbia      33
Pennsylvania          28
Quebec                24
Name: state_province, dtype: int64

7. Ahora nuestros jefes nos piden que saquemos las coordenadas de las provincias donde están ubicadas las universidades. Para eso nos piden que usemos la librería de geopy que aprendimos el día del repaso. Para desarrollar este ejercicio deberéis:
- Sacar los valores únicos de la columna state_province.
- Algunos de los valores que tenemos están con siglas, y deberéis reemplazarlos por lo siguiente:
    - NV: reemplazalo por Nevada
    - TX: reemplazalo por Texas
    - IN: reemplazalo por Indianapolis
    - CA: reemplazalo por California
    - VA: reemplazalo por Virginia
    - NY: reemplazalo por New York
    - MI: reemplazalo por Michigan
    - GA: reemplazalo por Georgia
    - ND: reemplazalo por North Dakota

In [21]:
df['state_province'].unique()

array(['Buenos Aires', 'Ciudad Autónoma de Buenos Aires', 'Entre Ríos',
       'Salta', 'Córdoba', 'Mendoza', 'Santa Fé', 'Unknown',
       'Santiago Del Estero', 'Misiones', 'Catamarca', 'Formosa', 'Jujuy',
       'La Rioja', 'La Pampa', 'San Juan', 'San Luis', 'Tucumán',
       'Quebec', 'Ontario', 'Nova Scotia', 'British Columbia', 'Alberta',
       'Manitoba', 'New Brunswick', 'Saskatchewan',
       'Newfoundland and Labrador', 'Prince Edward Island', 'Yukon',
       'Pennsylvania', 'NV', 'Iowa', 'VA', 'TX', 'Colorado', 'IN', 'CA',
       'South Carolina', 'Washington', 'NY', 'Texas', 'ND', 'MI', 'Ohio',
       'Florida', 'California', 'North Carolina', 'Michigan', 'GA',
       'New York, NY'], dtype=object)

In [22]:
dicc_estados = {'NV': 'Nevada',
                'TX': 'Texas',
                'IN': 'Indianapolis',
                'CA': 'California',
                'VA': 'Virginia',
                'NY': 'New York',
                'MI': 'Michigan',
                'GA': 'Georgia',
                'ND': 'North Dakota'}

In [23]:
def reemplazar_valores(dataframe, columna, diccionario):
    
    '''Acepta un dataframe, una columna y un diccionario y reemplaza 
    los valores de la columna indicada segun el diccionario.
    
    Parametros:
    - dataframe
    - string: nombre de una columna del dataframe
    - diccionario: los keys son los valores actuales y los values son los valores nuevos deseados
    
    Returns:
    - none
    '''
    
    try:
        for k, v in diccionario.items():
            dataframe[columna].replace(to_replace=k, value=v, inplace=True)
        
        print(f'Se han reemplazado los valores segun el diccionario. Ahora los valores unicos de la columna {columna} son:')
        print(dataframe['state_province'].unique())
        
    except:
        print('No se ha podido realizar la operación.')

In [24]:
reemplazar_valores(df, 'state_province', dicc_estados)

Se han reemplazado los valores segun el diccionario. Ahora los valores unicos de la columna state_province son:
['Buenos Aires' 'Ciudad Autónoma de Buenos Aires' 'Entre Ríos' 'Salta'
 'Córdoba' 'Mendoza' 'Santa Fé' 'Unknown' 'Santiago Del Estero' 'Misiones'
 'Catamarca' 'Formosa' 'Jujuy' 'La Rioja' 'La Pampa' 'San Juan' 'San Luis'
 'Tucumán' 'Quebec' 'Ontario' 'Nova Scotia' 'British Columbia' 'Alberta'
 'Manitoba' 'New Brunswick' 'Saskatchewan' 'Newfoundland and Labrador'
 'Prince Edward Island' 'Yukon' 'Pennsylvania' 'Nevada' 'Iowa' 'Virginia'
 'Texas' 'Colorado' 'Indianapolis' 'California' 'South Carolina'
 'Washington' 'New York' 'North Dakota' 'Michigan' 'Ohio' 'Florida'
 'North Carolina' 'Georgia' 'New York, NY']


- Otros valores que tenemos más formateados son y que deberemos reemplazar:
    - New York, NY. Deberéis reemplazarlo por "New York".
    - 'Buenos Aires', 'Ciudad Autónoma de Buenos Aires'. En este caso deberéis poner en ambos casos "Buenos Aires"

In [25]:
dicc_ciudades = {'Ciudad Autónoma de Buenos Aires':'Buenos Aires',
                 'New York, NY':'New York'}

In [26]:
reemplazar_valores(df, 'state_province', dicc_ciudades)

Se han reemplazado los valores segun el diccionario. Ahora los valores unicos de la columna state_province son:
['Buenos Aires' 'Entre Ríos' 'Salta' 'Córdoba' 'Mendoza' 'Santa Fé'
 'Unknown' 'Santiago Del Estero' 'Misiones' 'Catamarca' 'Formosa' 'Jujuy'
 'La Rioja' 'La Pampa' 'San Juan' 'San Luis' 'Tucumán' 'Quebec' 'Ontario'
 'Nova Scotia' 'British Columbia' 'Alberta' 'Manitoba' 'New Brunswick'
 'Saskatchewan' 'Newfoundland and Labrador' 'Prince Edward Island' 'Yukon'
 'Pennsylvania' 'Nevada' 'Iowa' 'Virginia' 'Texas' 'Colorado'
 'Indianapolis' 'California' 'South Carolina' 'Washington' 'New York'
 'North Dakota' 'Michigan' 'Ohio' 'Florida' 'North Carolina' 'Georgia']


- Una vez realizados los pasos anteriores, crea una lista con los valores únicos de las provincias de las universidades.
- Usando la API de geopy, extraed la latitud y la longitud de cada una de las provincias y almacenad los resultados en un dataframe.

In [27]:
lista_estado_prov = df['state_province'].unique().tolist()

In [28]:
def lat_long(lista):
    
    '''
    Acepta una lista de localidades y lo devuelve en un dataframe con los 
    latitudes y longitudes para cada localidad sacados con el API de geopy.
    
    Parametros:
    - lista de localidades
    
    Returns:
    - un dataframe con una columna de los localides, una columna de la latitud,
    y una columna de la longitud    
    '''
    
    diccionario ={'state_province':[], 'latitude': [], 'longitude': []}
    
    for i in lista:
        if i == 'Unknown':
            pass
        else:
            try:
                geolocator = Nominatim(user_agent='Cassia')
                location = geolocator.geocode(i)
                diccionario['state_province'].append(i)
                diccionario['latitude'].append(location[1][0]) 
                diccionario['longitude'].append(location[1][1]) 
                
            except:
                print(f'No se puede conseguir la latitud y longitud de {i}')
    
    return pd.DataFrame(diccionario)

In [29]:
df_coordinados = lat_long(lista_estado_prov)

In [30]:
df_coordinados.head(2)

,state_province,latitude,longitude
0,Buenos Aires,-34.607568,-58.437089
1,Entre Ríos,-31.625284,-59.353958


In [31]:
df_coordinados['state_province'].unique()

array(['Buenos Aires', 'Entre Ríos', 'Salta', 'Córdoba', 'Mendoza',
       'Santa Fé', 'Santiago Del Estero', 'Misiones', 'Catamarca',
       'Formosa', 'Jujuy', 'La Rioja', 'La Pampa', 'San Juan', 'San Luis',
       'Tucumán', 'Quebec', 'Ontario', 'Nova Scotia', 'British Columbia',
       'Alberta', 'Manitoba', 'New Brunswick', 'Saskatchewan',
       'Newfoundland and Labrador', 'Prince Edward Island', 'Yukon',
       'Pennsylvania', 'Nevada', 'Iowa', 'Virginia', 'Texas', 'Colorado',
       'Indianapolis', 'California', 'South Carolina', 'Washington',
       'New York', 'North Dakota', 'Michigan', 'Ohio', 'Florida',
       'North Carolina', 'Georgia'], dtype=object)

- Una vez que tengáis los datos del ejercicio anterior en un dataframe, unidlo con el de las universidades que hemos sacado de la API.

In [32]:
df = df.merge(df_coordinados, on='state_province', how='left')

In [33]:
# comprobamos que haya salido
df.head(2)

,state_province,name,country,web_pages,alpha_two_code,latitude,longitude
0,Buenos Aires,Universidad Atlantida Argentina,Argentina,http://www.atlantida.edu.ar/,AR,-34.607568,-58.437089
1,Buenos Aires,Universidad Austral Buenos Aires,Argentina,http://www.austral.edu.ar/,AR,-34.607568,-58.437089


8. Crea una BBDD en mysql que contenga las siguientes tablas:
- Tabla países: donde encontraremos las siguientes columnas:
    - idestado: primary key, integer, autoincremental
    - nombre_pais: varchar
    - nombre_provincia: varchar
    - latitud: decimal
    - longitud: decimal
- Tabla universidades: donde encontraremos las siguientes columnas:
    - iduniversidades: primary key, integer, autoincremental
    - nombre_universidad: varchar
    - agina_web: varchar
    - paises_idestado: foreing key

In [34]:
def crear_bbdd(nombre_bbdd, contraseña):
    
    '''
    Acepta dos strings, un nombre para crear una base de datos en MySQL, y la contraseña
    de MySQL para connectar con MySQL connector. Si la conexion falla o si la base de datos ya existe,
    devuelve un mensaje de error.
    
    Parametros:
    - string del nombre que poner a la base de datos
    - string con la contraseña de MySQL
    
    Returns:
    - none
    '''
    
    conexion = mysql.connector.connect(host="localhost",
                                    user="root",
                                    password=contraseña)
                                    
    print("La conexión a MySQL se ha realizado con exito.")
    
    cursor = conexion.cursor()

    try:
        cursor.execute(f"CREATE DATABASE IF NOT EXISTS {nombre_bbdd};")
        print(cursor)
    except mysql.connector.Error as err:
        print(err)
        print("Error Code:", err.errno)
        print("SQLSTATE", err.sqlstate)
        print("Message", err.msg)

In [35]:
crear_bbdd('universidades', 'AlumnaAdalab')

La conexión a MySQL se ha realizado con exito.
CMySQLCursor: CREATE DATABASE IF NOT EXISTS universida..


In [36]:
def crear_insertar_tabla(nombre_bbdd, contraseña, query):
    
    '''
    Acepta tres strings: el nombre de la base de datos, la contraseña de MySQL y una
    query para crear una tabla en la base de datos indicado o para insertar datos en una
    tabla de esa bbdd. Si la conexion falla o si la tabla ya existe, devuelve un mensaje de error.
    
    Parametros:
    - string: nombre de la base de datos
    - string: contraseña para MySQL
    - string: query de creacion de tabla o insercion de datos
    
    Returns:
    - none
    '''
    
    conexion = mysql.connector.connect(user='root', password=contraseña,
                                     host='127.0.0.1', database=nombre_bbdd)
    cursor = conexion.cursor()
    
    try: 
        cursor.execute(query)
        conexion.commit() 

    except mysql.connector.Error as err:
        print(err)
        print("Error Code:", err.errno)
        print("SQLSTATE", err.sqlstate)
        print("Message", err.msg)

In [37]:
query_tabla_paises = """CREATE TABLE IF NOT EXISTS `universidades`.`paises` (
                        `idestado` INT NOT NULL AUTO_INCREMENT,
                        `nombre_pais` VARCHAR(45) NOT NULL,
                        `nombre_provincia` VARCHAR(45),
                        `latitud` VARCHAR(45) NULL,
                        `longitud` VARCHAR(45) NULL,
                        PRIMARY KEY (`idestado`))
                        ENGINE = InnoDB;"""

In [38]:
query_tabla_universidades = """CREATE TABLE IF NOT EXISTS `universidades`.`universidades` (
                            `iduniversidades` INT NOT NULL AUTO_INCREMENT,
                            `nombre_universidad` VARCHAR(100) NOT NULL,
                            `pagina_web` VARCHAR(100) NULL,
                            `idestado` INT NOT NULL,
                            PRIMARY KEY (`iduniversidades`),
                            INDEX `fk_universidades_paises_idx` (`idestado` ASC) VISIBLE,
                            CONSTRAINT `fk_universidades_paises`
                                FOREIGN KEY (`idestado`)
                                REFERENCES `universidades`.`paises` (`idestado`)
                                ON DELETE CASCADE
                                ON UPDATE CASCADE)
                            ENGINE = InnoDB;"""

In [39]:
crear_insertar_tabla('universidades', 'AlumnaAdalab', query_tabla_paises)

In [40]:
crear_insertar_tabla('universidades', 'AlumnaAdalab', query_tabla_universidades)

9. Introduce todo el código que habéis ido creando en funciones, siguiendo la misma lógica que hemos seguido en los pairs

In [41]:
# ya esta casi todo el codigo en funciones; solo falta el merge con el dataframe de latitud y longitud
# yo lo metería junto con el codigo de geopy:

def lat_long_merge(dataframe, user_agent):
    
    '''
    Acepta un dataframe de datos de universidades del API de hipolabs y el
    nombre de user agent para usar con el API de geopy. Devuelve el dataframe 
    con dos columnas nuevas indicando la latitud y la longitud del estado o 
    provincia de cada universidad sacados con el API de geopy.
    
    Parametros:
    - dataframe de datos de universidades del API de hipolabs
    - string: un nombre para usar como user agent con el API de geopy
    
    Returns:
    - el dataframe origina con una columna para la latitud y una columna para la longitud    
    '''
    
    lista_prov = dataframe['state_province'].unique().tolist()
    
    diccionario ={'state_province':[], 'latitude': [], 'longitude': []}
    
    for i in lista_prov:
        if i == 'Unknown' or i is np.nan:
            pass
        else:
            try:
                geolocator = Nominatim(user_agent=user_agent)
                location = geolocator.geocode(i)
                diccionario['state_province'].append(i)
                diccionario['latitude'].append(location[1][0]) 
                diccionario['longitude'].append(location[1][1]) 
                
            except:
                print(f'No se puede conseguir la latitud y longitud de {i}')
    
    df_lat_long = pd.DataFrame(diccionario)
    
    return dataframe.merge(df_lat_long, on='state_province', how='left')

10. BONUS
- Introduce los datos en la BBDD de SQL.

In [42]:
# creamos un nuevo dataframe sin duplicados de pais/provincia
df_paises = df.drop_duplicates(subset=['state_province', 'country'])

In [43]:
# insertamos los datos de la tabla de paises
for indice, fila in df_paises.iterrows():

    query_insert_paises = f"""INSERT INTO paises (nombre_pais, nombre_provincia, latitud, longitud)  
        VALUES ('{fila["country"]}', '{fila["state_province"]}', '{fila["latitude"]}', '{fila["longitude"]}');"""
    
    crear_insertar_tabla("universidades", "AlumnaAdalab", query_insert_paises)

In [44]:
def devolver_datos(nombre_bbdd, contraseña, query):
    
    ''' 
    Acepta el nombre de una base de datos, la contraseña de MySQL, y una query de SELECT de MySQL, y devuelve un
    dataframe con los resultados del query. Si no funcion devuelve un mensaje de error.
    
    Parametros:
    - string: nombre de una base de datos existente en MySQL
    - string: contraseña de MySQL
    - string: query tipo SELECT de MySQL
    '''
    
    conexion = mysql.connector.connect(user='root', password=contraseña,
                                     host='127.0.0.1', database=nombre_bbdd)
    
    try: 
        return pd.read_sql_query(query, conexion)
    
    except mysql.connector.Error as err:
        print(err)
        print("Error Code:", err.errno)
        print("SQLSTATE", err.sqlstate)
        print("Message", err.msg)

In [57]:
# saco los datos de la tabla creada en MySQL para obtener el idestado
df_mysql_paises = devolver_datos('universidades', 'AlumnaAdalab', 'SELECT * FROM `paises`')
df_mysql_paises.drop(columns=['nombre_pais','latitud', 'longitud'], inplace=True)
df_mysql_paises.columns = ['idestado','state_province']
df_mysql_paises.head(2)

/tmp/ipykernel_7184/1694167056.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql_query(query, conexion)


,idestado,state_province
0,1,Buenos Aires
1,2,Entre Ríos


In [58]:
df3 = df.copy()

In [59]:
# junto los datos de la tabla de paises de MySQL con una copia del dataframe
df3 = df3.merge(df_mysql_paises, on='state_province', how='left')

In [60]:
# elimino las filas duplicadas creadas en el merge y reseteo los indices para que esten en orden
df3.drop_duplicates(subset=['name', 'state_province'], inplace=True)
df3.reset_index(drop=True, inplace=True)
df3

,state_province,name,country,web_pages,alpha_two_code,latitude,longitude,idestado
0,Buenos Aires,Universidad Atlantida Argentina,Argentina,http://www.atlantida.edu.ar/,AR,-34.607568,-58.437089,1
1,Buenos Aires,Universidad Austral Buenos Aires,Argentina,http://www.austral.edu.ar/,AR,-34.607568,-58.437089,1
2,Buenos Aires,"Universidad CAECE, Buenos Aires",Argentina,http://www.caece.edu.ar/,AR,-34.607568,-58.437089,1
3,Buenos Aires,Instituto Universitario CEMA,Argentina,http://www.cema.edu.ar/,AR,-34.607568,-58.437089,1
4,Buenos Aires,Instituto de Enseñanza Superior del Ejército,Argentina,http://www.iese.edu.ar/,AR,-34.607568,-58.437089,1
...,...,...,...,...,...,...,...,...
2502,Unknown,Virginia University of Lynchburg,United States,https://www.vul.edu/,US,NaN,NaN,7
2503,Unknown,Voorhees University,United States,https://www.voorhees.edu/,US,NaN,NaN,7
2504,Unknown,West Virginia State University,United States,https://www.wvstateu.edu/,US,NaN,NaN,7
2505,Unknown,Wiley College,United States,https://www.wileyc.edu/,US,NaN,NaN,7


In [49]:
# quito comillas de los valores del dataframe por si da errores de syntaxes de MySQL
df.replace(r'"*', '', regex=True, inplace=True)
df.replace(r"'*", "", regex=True, inplace=True)

In [50]:
# inserto los datos de la tabla de universidades
# no funciona pero dejo el código para mostrar que lo he intentado

for indice, fila in df3.iterrows():

    query_insert_unis = f'''INSERT INTO universidades (nombre_universidad, pagina_web, idestado)  
        VALUES ("{fila['name']}", "{fila['web_pages']}", "{fila['idestado']}");'''
    
    crear_insertar_tabla("universidades", "AlumnaAdalab", query_insert_unis)

1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'John F. Kennedy"", "http://www.kennedy.edu.ar/", "1")' at line 2
Error Code: 1064
SQLSTATE 42000
Message You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'John F. Kennedy"", "http://www.kennedy.edu.ar/", "1")' at line 2
1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'Juan Agustín Maza"", "http://www.umaza.edu.ar/", "7")' at line 2
Error Code: 1064
SQLSTATE 42000
Message You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near 'Juan Agustín Maza"", "http://www.umaza.edu.ar/", "7")' at line 2
1064 (42000): You have an error in your SQL syntax; check the manual that corresponds to you

KeyboardInterrupt: 